In [ ]:
from map import Map, Marker
import db

In [ ]:
locations = db.query(
    """
select round(st_x(geom), 5), round(st_y(geom), 6), count(*) as num_events

from chicago_escooter_events

group by 1,2
having num_events > 10
order by 3 desc
"""
)

In [ ]:
m = Map((41.85, -87.5))
for row in locations.fetchall():
    m.markers.append(Marker((row[1], row[0]), f"Num Events: {row[2]}", "gear"))
m

In [ ]:
db.query(
    """
select vendor, max(start_time) as start_time, max(end_time) as end_time, count(*) num_trips, sum(trip_distance) total_trip_distance

from raw.chicago_escooter_trips

group by 1
order by num_trips desc
"""
)

In [ ]:
db.query(
    """
    select 
    community_area_name, count(*) as num_events
    from chicago_escooter_events
    group by 1
    having num_events > 50
    order by 2 desc
"""
).df().plot.bar(
    x="community_area_name", title="Monthly Station Events", rot=45, figsize=(20, 5)
)

In [ ]:
db.query(
    """
 select 
    strftime(event_at, '%Y-%m') as month
    , count(*) as num_events
    from chicago_escooter_events e
    group by 1
    order by month asc
"""
).df().plot.bar(x="month", title="Monthly Events", rot=45)

In [ ]:
out_of_bounds = db.query(
    """
select st_x(e.geom), st_y(e.geom), e.trip_id
from chicago_escooter_events e
left join boundaries w on st_contains(w.geom, e.geom)
where w.geom is null
limit 1000
"""
)
out_of_bounds

In [ ]:
boundaries = db.query(
    """
select st_asgeojson(geom) from boundaries
where id = 'chicago' 
"""
)

In [ ]:
from ipyleaflet import GeoJSON
import json

m = Map((41.84, -87.5))
for row in out_of_bounds.fetchall():
    m.markers.append(Marker((row[1], row[0]), f"Num Events: {row[2]}", "question"))

for row in boundaries.fetchall():
    geo_json = GeoJSON(data=json.loads(row[0]), style={"color": "blue", "weight": 1})
    m.add(geo_json)
m

In [ ]:
db.query(
    """
select * from raw.chicago_escooter_trips
limit 10
"""
)